In [144]:
import sys
# !{sys.executable} -m pip install lyricsgenius
# !{sys.executable} -m pip install spotipy
# !{sys.executable} -m pip install selenium
import pandas as pd
import requests
from urllib.request import quote
import lyricsgenius
import spotipy
import spotipy.oauth2 as oauth2
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import json
import string
from random import *

def _new_format_api_request(self, term_and_type, page=1):
    request_term, request_type = str(term_and_type[0]), term_and_type[1]
    assert request_type in self._API_REQUEST_TYPES, "Unknown API request type"
    if request_type=='artist-songs':
        return self._API_URL + 'artists/' + quote(request_term) + '/songs?sort=popularity&page=' + str(page)
    else:        
        return self._API_URL + self._API_REQUEST_TYPES[request_type] + quote(request_term)
    
lyricsgenius.api._API._format_api_request = _new_format_api_request

In [122]:
genius_client_id = 'Qeii-UKMUUwsAYCuuemsF7CZk7R2bFDhjP1FMkqpozASvYsA4wQDlLmd2JFM3Vhm'
genius_secret_id = 'fWhV7VtwAL-5D32Ga9Y0Kyh_VFEo7XK0Jf8-3RMcZ5EoMXqsDpmFWFFPyobnibfMXJuJ-GZe69YEuXR-36VGAA'
genius_client_access_token = '1QVL91jZbgdOIhvYhqKku_zkFDf6MHEyLpXEITHU2tE2HGvAqYIPInTrQaWcInD7'
app_website_url = 'http://cis545genius.com'
redirect_uri = 'cis545genius.com'
base_url = 'https://api.genius.com'
driver_path = '/Users/ericdong/Downloads/chromedriver'
# driver_path = "/mnt/c/Users/jeffr/Downloads/chromedriver_win32/chromedriver.exe"
artist_url = 'https://www.ranker.com/crowdranked-list/the-greatest-rappers-of-all-time'
page_downs = 100
all_genius_artists = 'https://genius.com/artists-index/'
api = lyricsgenius.Genius(genius_client_access_token)

# eric's creds
spotify_client_id = '96bdc763e3b5446ebbd6c44850055bcb'
spotify_secret_id = 'f7a5054823ed491aa1856ecdc7849062'
spotify_uri = 'http://cis545spotify.com'
# jeffrey's creds
# spotify_client_id = '1e482b4980804a4a95e1f213a1780763'
# spotify_secret_id = 'd624a1a562294382b9af1de69983c452'
# spotify_token = 'BQDzi63yeLiTqICzNmT4FSmhV0PihaHEkrD9Bz-sBr78-y8_z_lBvlJi_ch1omJ8d9MCwjJuT_xlPXiO7wqaRrKu6FefifZ9DxHpWXXIQ48Zvm_y6UapbJMn74J3r2VFgCxnQ5FcqWk'

In [3]:
def get_artists(url, page_downs, driver_path, letter):
    url = url + str(letter)
    driver = webdriver.Chrome(driver_path)
    driver.get(url)
    time.sleep(1)
#     for i in range(page_downs):
#         driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#         time.sleep(0.2)
    artists = driver.find_elements_by_class_name('artists_index_list-artist_name')
    names = [artist.text for artist in artists]
    driver.quit()
    
    return names

In [9]:
def get_songs(artist):
    return api.search_artist(artist, verbose=False, max_songs=10)

In [5]:
letters = list(string.ascii_lowercase) + [0]
artists = []
for letter in letters:
    artists = artists + get_artists(all_genius_artists, page_downs, driver_path, letter=letter)

In [6]:
print(len(artists))
print(artists)

540
['Żabson', 'Achille Lauro', 'Adele', 'АИГЕЛ (AIGEL)', 'Alan Menken', 'Alessia Cara', 'Alicia Keys', 'alt-J', 'Amy Winehouse', 'Anderson .Paak', 'Andrew Lloyd Webber', 'Arctic Monkeys', 'Ariana Grande', 'A$AP Ferg', 'A$AP Mob', 'A$AP Rocky', 'ATL', 'at Wendys', 'Azealia Banks', 'Azet', 'Bad Bunny', 'Bazzi', 'The Beatles', 'Ben and Ben', 'Beyoncé', 'Bhad Bhabie', 'Big Sean', 'Big Shaq', 'Billie Eilish', 'blackbear', 'BLACKPINK', 'BlocBoy JB', 'Bob Dylan', 'Bones', 'Booba', 'A Boogie Wit Da Hoodie', 'BROCKHAMPTON', 'Bruno Mars', 'Bryson Tiller', 'BTS', 'Calum Scott', 'Calvin Harris', 'Camila Cabello', 'Cardi B', 'Car Seat Headrest', 'Cast - ZOMBIES', 'Ceza', 'The Chainsmokers', 'Chance The Rapper', 'Charlie Puth', 'Chief Keef', 'Childish Gambino', 'Chloe x Halle', 'Chris Brown', 'Chris Brown & Joyner Lucas', 'Cigarettes After Sex', 'Clean Bandit', 'Coldplay', 'CUCO', 'CupcakKe', 'Dadju', 'Damso', 'Daniel Caesar', 'Dave', 'David Bowie', 'DBangz', 'Death Grips', 'Demi Lovato', 'Denzel C

In [37]:
# test set of artists
# artists = ['Kanye West', 'Migos', 'Drake']
# print(artists.index('Joey Bada$$ & XXXTENTACION'))
# print(artists.index('Machine Gun Kelly'))
# print(len(new_new_new_music))
# temp_music = music + new_new_new_music[1:] + new_music[10:] + new_new_music[10:]
# print(len(temp_music))
# music = temp_music

In [25]:
music = []
for artist in artists:
    try:
        music = music + get_songs(artist).songs
    except AssertionError:
        pass
# get_songs('Kanye West').songs

In [32]:
# print(len(music))
# print(music)

In [33]:
# print(new_new_new_music)

In [34]:
# print(len(new_music))
# print(new_music)

In [35]:
# print(len(new_new_music))
# print(new_new_music)

In [36]:
# music = list(filter(lambda x: x.__dict__['media'], music))
# kept_music = list(filter(lambda x: x.title in list(df.song) and x.artist in list(df.artist), music))
# df['genius_id'] = list(map(lambda x: x.__dict__['_api_path'].split('/')[-1], kept_music))

In [163]:
print(len(music))

10653


In [162]:
# headers = {'Authorization': 'Bearer %s' % genius_client_access_token}
# search_url = base_url + '/search'
# song_title = 'Mercy'
# params = {'q': song_title}
# response = requests.get(search_url, params=params, headers=headers)
# json = response.json()
# print(json)
while len(music) < 12500:
    old_ids = list(map(lambda x: x.__dict__['_body']['id'], music))
    rand_id = randint(0, 9999999)
    headers = {'Authorization': 'Bearer %s' % genius_client_access_token}
    params = {'id': rand_id}
    search_url = base_url + '/songs/' + str(rand_id)
    response = requests.get(search_url, params=params, headers=headers).json()
    if response['meta']['status'] == 200:
        if response['response']['song']['id'] not in old_ids:
            song_name = response['response']['song']['title']
            song_artist = response['response']['song']['primary_artist']['name']
            random_search = api.search_song(song_name, song_artist, verbose=False)
            if random_search is not None:
                music.append(random_search)

KeyboardInterrupt: 

In [98]:
# music.append(api.search_song('Power', 'Kanye West', verbose=False))
# music.append(api.search_song('Energy', 'Drake', verbose=False))
# music.append(api.search_song('Mask Off', 'Future', verbose=False))
# music.append(api.search_song('Mercy', 'Kanye West', verbose=False))

In [101]:
# def remove_duplicates(seq):
#     seen = set()
#     seen_add = seen.add
#     return [x for x in seq if not (x in seen or seen_add(x))]
# music = remove_duplicates(music)
# music = [x for x in music if x is not None]
# print(len(set(music)))
# print(len(music))
# for x in music:
#     if x is None:
#         print(x)

In [177]:
df = pd.DataFrame()
df['artist'] = list(map(lambda x: x.artist, music))
df['song'] = list(map(lambda x: x.title, music))
df['album'] = list(map(lambda x: x.album, music))
df['lyrics'] = list(map(lambda x: x.lyrics, music))
df['genius_id'] = list(map(lambda x: x.__dict__['_body']['id'], music))
# df['spotify_id'] = list(map(lambda x: x.__dict__['media'], music))
# df['genius_id'] = list(map(lambda x: x.__dict__['_api_path'].split('/')[-1], kept_music))

In [103]:
# list(map(lambda x: x.annotation, music))
# print(type(music[0]))
# print(music[0].__dict__['_body'])

# import requests
# from bs4 import BeautifulSoup
# import json

# def get_annotations(song):
#     print(song.title)
#     api_calls = [annotation['api_path'] for annotation in
#                  song.__dict__['_body']['description_annotation']['annotations']]
# #     print(song.__dict__.keys())
# #     print(song.__dict__)
# #     print(song.__dict__['_body'])
#     #     print(api_calls)
#     def get_from_api(api_url):
#         print('http://api.genius.com' + api_url)
#         page = requests.get('http://api.genius.com' + api_url,
#                             headers={'Authorization': 'Bearer ' + genius_client_access_token})
#         print(page.__dict__)
# #         print(page.__dict__.keys())
#     #         annotation_dict = json.loads(page.__dict__['_content'].decode())['response']['annotation']['body']['dom']
#     # #         print(annotation_dict)
#     #         for key in annotation_dict:
#     #             print("NEW")
#     #             print(key)
#     #             print(annotation_dict[key])
#     return [get_from_api(url) for url in api_calls]
# get_annotations(music[2])

In [178]:
df.to_pickle('data.pkl')
df

,artist,song,album,lyrics,genius_id
0,Żabson,DMT,To Ziomal,"Ej, ej\n\nŻycie jest jak kalambury, wiem co po...",3480937
1,Żabson,Incepcja,To Ziomal,"Kruszę w dłoni stuff, w majtkach mam go więcej...",3382127
2,Żabson,Księżniczki,To Ziomal,Dzisiaj olewam te księżniczki bo dzisiaj inacz...,3199547
3,Żabson,Czarne okulary,Passion Fruits EP,Zapomniałem już o tamtych snach\nBo wszystko m...,2451239
4,Żabson,Do ziomów,NieKumam,"Geezy, Geezy\n\nO swoje szczęście nie chcę wie...",666753
5,Żabson,Pok0ra,NieKumam,"Frog-son, elo\n (x2)\nMówią wciąż ""żaby styl j...",622858
6,Żabson,Rich Art,None,Ja ciągle kocham nawijać te zwrotki\nChoć już ...,2922815
7,Żabson,Biznesmen,Passion Fruits EP,Jestem bohaterem w tym mieście\nMam tylko szcz...,2650342
8,Żabson,BubbleGum Free,BubbleGum Free,"Bubble, bubble, hy, hy, hy, bubble\nBubble, bu...",2848399
9,Żabson,Skunax Free,Passion Fruits EP,Nigdy nie zobaczysz u mnie łezki w oku\nPalę s...,2477314


In [167]:
# spotify = spotipy.Spotify(auth=spotify_token)
# def get_song(song):
    
# spotify_artists = [spotify.search(q=song.title, type='track') for song in music]

sp_oauth = oauth2.SpotifyOAuth(client_id=spotify_client_id, client_secret=spotify_secret_id, 
                               redirect_uri=spotify_uri)
token_info = sp_oauth.get_cached_token() 
if not token_info:
    auth_url = sp_oauth.get_authorize_url()
    print(auth_url)
    response = input('Paste the above link into your browser, then paste the redirect url here: ')

    code = sp_oauth.parse_response_code(response)
    token_info = sp_oauth.get_access_token(code)

    token = token_info['access_token']

spotify = spotipy.Spotify(auth=token)

https://accounts.spotify.com/authorize?client_id=96bdc763e3b5446ebbd6c44850055bcb&response_type=code&redirect_uri=http%3A%2F%2Fcis545spotify.com
Paste the above link into your browser, then paste the redirect url here: http://cis545spotify.com/?code=AQBh_XyafXvg_o70cHyZQ6lcyx6owM8o123b10iqSHQAwz6aDFv6MIv8_1DGMDJ3sUn7wui6Scsqcs1hCV4qZqG6k2F2aktSW7oyngyP2bxjea2ogqxPykE1aGW5AKomeBEwMHLqifyTUQVPt0H4XieB7_KDGgEIZq6Vvy-dLtHnnzt1I8IB-TihWcDWEQ


In [147]:
# print(music[1])
# print(music[1].title)
# print(spotify_artists[1])
# name = 'Best I Ever Had'
# artist = 'Drake'
# results = spotify.search(q='track:' + name, limit=50, type='track')
# items = results['tracks']['items']
# # string1.lower() == string2.lower()
# # items[0]['artists'][0]['name']
# right = next(item for item in items if 
#            item['name'].lower() == name.lower() and item['artists'][0]['name'].lower() == artist.lower())
# print(right)

In [151]:
# yeezy = spotify.artist_top_tracks('5K4W6rqBFWDnAN6FQUkS6x')
# for track in yeezy['tracks'][:10]:
#     print('track    : ' + track['name'])

In [179]:
table = str.maketrans(dict.fromkeys('()[]-*'))

def is_token_expired(token_info):
    now = int(time.time())
    return token_info['expires_at'] - now < 60

def refresh():
    global token_info, spotify

    if is_token_expired(token_info):
        token_info = sp_oauth.refresh_access_token(token_info['refresh_token'])
        token = token_info['access_token']
        spotify = spotipy.Spotify(auth=token)

def search_song(row):
    refresh()
    artist = row[0]
    song = row[1]
    results = spotify.search(q='track:' + song, limit=20, type='track')
    items = results['tracks']['items']
    if len(items) != 0:
        result = next((item for item in items if 
                       item['name'].lower().translate(table) == song.lower().translate(table) and 
                       artist.lower() in item['artists'][0]['name'].lower()), 
                       None)
        if result is None:
            return None
        return result['id']
    else:
        return None

In [185]:
partial_df = df[:5500]
# partial_df = df[5500:]
# df[0:5000]

In [ ]:
partial_df['spotify_id'] = partial_df.apply(search_song, axis=1)

In [158]:
df.dropna(axis=0, subset=['spotify_id'], inplace=True)
df['track_popularity'] = df.apply(lambda row: spotify.track(row.spotify_id)['popularity'], axis=1)
df['duration'] = df.apply(lambda row: spotify.track(row.spotify_id)['duration_ms'], axis=1)
df['explicit'] = df.apply(lambda row: spotify.track(row.spotify_id)['explicit'], axis=1)
df['danceability'] = df.apply(lambda row: spotify.audio_features(row.spotify_id)[0]['danceability'], axis=1)
df['energy'] = df.apply(lambda row: spotify.audio_features(row.spotify_id)[0]['energy'], axis=1)
df['key'] = df.apply(lambda row: spotify.audio_features(row.spotify_id)[0]['key'], axis=1)
df['loudness'] = df.apply(lambda row: spotify.audio_features(row.spotify_id)[0]['loudness'], axis=1)
df['mode'] = df.apply(lambda row: spotify.audio_features(row.spotify_id)[0]['mode'], axis=1)
df['speechiness'] = df.apply(lambda row: spotify.audio_features(row.spotify_id)[0]['speechiness'], axis=1)
df['acousticness'] = df.apply(lambda row: spotify.audio_features(row.spotify_id)[0]['danceability'], axis=1)
df['instrumentalness'] = df.apply(lambda row: spotify.audio_features(row.spotify_id)[0]['instrumentalness'], axis=1)
df['liveness'] = df.apply(lambda row: spotify.audio_features(row.spotify_id)[0]['liveness'], axis=1)
df['valence'] = df.apply(lambda row: spotify.audio_features(row.spotify_id)[0]['valence'], axis=1)
df['tempo'] = df.apply(lambda row: spotify.audio_features(row.spotify_id)[0]['tempo'], axis=1)
df['time_signature'] = df.apply(lambda row: spotify.audio_features(row.spotify_id)[0]['time_signature'], axis=1)

In [ ]:
df['album'] = df.apply(lambda row: spotify.track(row.spotifyid)['album']['name'], axis=1)
def featured_artists(artist_names):
    if len(artist_names) > 1:
        names = list(map(lambda x: x['name'], artist_names))
        return names[1:]
    else:
        return []
    
df['featured_artists'] = df.apply(lambda row: featured_artists(spotify.track(row.spotify_id)['artists']), 
                                  axis=1)

In [156]:
# print(spotify.audio_features('3fyMH1t6UPeR5croea9PrR')[0]['danceability'])
# print(spotify.track('3fyMH1t6UPeR5croea9PrR')['duration_ms'])

In [180]:
df['lyrics'] = df['lyrics'].apply(lambda x: x.split())

In [181]:
df.to_pickle('data.pkl')

In [186]:
df.to_csv('data.csv')

In [183]:
df

,artist,song,album,lyrics,genius_id
0,Żabson,DMT,To Ziomal,"[Ej,, ej, Życie, jest, jak, kalambury,, wiem, ...",3480937
1,Żabson,Incepcja,To Ziomal,"[Kruszę, w, dłoni, stuff,, w, majtkach, mam, g...",3382127
2,Żabson,Księżniczki,To Ziomal,"[Dzisiaj, olewam, te, księżniczki, bo, dzisiaj...",3199547
3,Żabson,Czarne okulary,Passion Fruits EP,"[Zapomniałem, już, o, tamtych, snach, Bo, wszy...",2451239
4,Żabson,Do ziomów,NieKumam,"[Geezy,, Geezy, O, swoje, szczęście, nie, chcę...",666753
5,Żabson,Pok0ra,NieKumam,"[Frog-son,, elo, (x2), Mówią, wciąż, ""żaby, st...",622858
6,Żabson,Rich Art,None,"[Ja, ciągle, kocham, nawijać, te, zwrotki, Cho...",2922815
7,Żabson,Biznesmen,Passion Fruits EP,"[Jestem, bohaterem, w, tym, mieście, Mam, tylk...",2650342
8,Żabson,BubbleGum Free,BubbleGum Free,"[Bubble,, bubble,, hy,, hy,, hy,, bubble, Bubb...",2848399
9,Żabson,Skunax Free,Passion Fruits EP,"[Nigdy, nie, zobaczysz, u, mnie, łezki, w, oku...",2477314


In [205]:
# music[-1].__dict__['_api_path'].split('/')[-1]
# music[-1].__dict__['_body']['media']
# kept_music = list(filter(lambda x: x.title in list(df.song) and x.artist in list(df.artist), music))
# df['genius_id'] = list(map(lambda x: x.__dict__['_api_path'].split('/')[-1], kept_music))